In [1]:
import pandas as pd

# For Alabama temperature data
al_temp = pd.read_csv('../datasets/state_temperature_data/ohio_cleaned_temperature_data.csv')
print("\nAlabama Temperature Data Types:")
print(al_temp.dtypes)
print("\nFirst few rows of Alabama Temperature:")
print(al_temp.head())

# For electricity usage data
elec_usage = pd.read_csv('../datasets/cleaned_electricity_usage_data.csv')
print("\nElectricity Usage Data Types:")
print(elec_usage.dtypes)
print("\nFirst few rows of Alabama Electricity Usage:")
print(elec_usage[elec_usage['stateDescription'] == 'Alabama'].head())


Alabama Temperature Data Types:
year       int64
month      int64
tavg     float64
dtype: object

First few rows of Alabama Temperature:
   year  month  tavg
0  2001      1  28.0
1  2001      2  32.0
2  2001      3  42.0
3  2001      4  52.0
4  2001      5  62.5

Electricity Usage Data Types:
year                  int64
month                 int64
stateDescription     object
sectorName           object
price               float64
revenue             float64
sales               float64
dtype: object

First few rows of Alabama Electricity Usage:
     year  month stateDescription   sectorName  price    revenue       sales
10   2001      1          Alabama  all sectors   5.54  407.61261  7362.47302
53   2001      2          Alabama  all sectors   5.31  321.06715  6041.02574
110  2001      3          Alabama  all sectors   5.87  345.77802  5894.61038
173  2001      4          Alabama  all sectors   5.72  347.18634  6064.53539
248  2001      5          Alabama  all sectors   5.60  359.09236

In [8]:
# First load both datasets
elec_df = pd.read_csv('../datasets/cleaned_electricity_usage_data.csv')
al_temp = pd.read_csv('../datasets/state_temperature_data/alabama_cleaned_temperature_data.csv')

# Since we want to merge with state name, let's add state column to temperature data
al_temp['stateDescription'] = 'Alabama'

# Merge datasets for Alabama only
# This keeps all electricity columns and adds temperature at the end
test_merge = pd.merge(
    elec_df[elec_df['stateDescription'] == 'Alabama'],
    al_temp[['year', 'month', 'stateDescription', 'tavg']],
    on=['year', 'month', 'stateDescription'],
    how='left'
)

# Rename the temperature column
test_merge = test_merge.rename(columns={'tavg': 'average temperature in Fahrenheit'})

test_merge.to_csv('../datasets/test.csv', index=False)

In [2]:
import pandas as pd
import glob
import os

# 1. Read the main electricity usage data
elec_df = pd.read_csv('../datasets/cleaned_electricity_usage_data.csv')

# 2. Get list of all state temperature files
state_files = glob.glob('../datasets/state_temperature_data/*_cleaned_temperature_data.csv')

# Print to check if we're finding the files
print(f"Found {len(state_files)} state files:")
print(state_files[:5])  # Print first 5 files to verify

# 3. Create empty list to store merged data for each state
merged_states = []

# 4. Loop through each state file
for state_file in state_files:
    # Extract state name from file name
    # Handle compound state names properly
    state_file_name = os.path.basename(state_file)
    state_name = state_file_name.split('_cleaned_temperature_data.csv')[0].replace('_', ' ').title()
    print(f"\nProcessing state: {state_name}")
    
    # Read state's temperature data
    state_temp = pd.read_csv(state_file)
    print(f"Temperature data shape: {state_temp.shape}")
    
    # Print to verify we have matching records in electricity data
    state_elec = elec_df[elec_df['stateDescription'] == state_name]
    print(f"Electricity data shape for {state_name}: {state_elec.shape}")
    
    # Add state name to temperature data
    state_temp['stateDescription'] = state_name
    
    # Merge this state's data
    state_merged = pd.merge(
        state_elec,
        state_temp[['year', 'month', 'stateDescription', 'tavg']],
        on=['year', 'month', 'stateDescription'],
        how='left'
    )
    print(f"Merged shape: {state_merged.shape}")
    
    # Add to our list of merged states
    merged_states.append(state_merged)

print(f"\nTotal states processed: {len(merged_states)}")

Found 50 state files:
['../datasets/state_temperature_data\\alabama_cleaned_temperature_data.csv', '../datasets/state_temperature_data\\alaska_cleaned_temperature_data.csv', '../datasets/state_temperature_data\\arizona_cleaned_temperature_data.csv', '../datasets/state_temperature_data\\arkansas_cleaned_temperature_data.csv', '../datasets/state_temperature_data\\california_cleaned_temperature_data.csv']

Processing state: Alabama
Temperature data shape: (287, 3)
Electricity data shape for Alabama: (277, 7)
Merged shape: (277, 8)

Processing state: Alaska
Temperature data shape: (287, 3)
Electricity data shape for Alaska: (277, 7)
Merged shape: (277, 8)

Processing state: Arizona
Temperature data shape: (287, 3)
Electricity data shape for Arizona: (277, 7)
Merged shape: (277, 8)

Processing state: Arkansas
Temperature data shape: (287, 3)
Electricity data shape for Arkansas: (277, 7)
Merged shape: (277, 8)

Processing state: California
Temperature data shape: (286, 3)
Electricity data sh

In [3]:
# 5. Combine all states into one DataFrame
final_df = pd.concat(merged_states, ignore_index=True)

# 6. Sort the data by state, year, and month
final_df = final_df.sort_values(['stateDescription', 'year', 'month'])

# 7. Save to new CSV file
final_df.to_csv('../datasets/merged_electricity_temperature_data.csv', index=False)

# 8. Print some information about the final dataset
print("\nFinal merged dataset info:")
print(f"Total number of records: {len(final_df)}")
print(f"Number of states: {final_df['stateDescription'].nunique()}")
print(f"Date range: {final_df['year'].min()}-{final_df['month'].min()} to {final_df['year'].max()}-{final_df['month'].max()}")
print("\nFirst few rows:")
print(final_df.head())


Final merged dataset info:
Total number of records: 13850
Number of states: 50
Date range: 2001-1 to 2024-12

First few rows:
   year  month stateDescription   sectorName  price    revenue       sales  \
0  2001      1          Alabama  all sectors   5.54  407.61261  7362.47302   
1  2001      2          Alabama  all sectors   5.31  321.06715  6041.02574   
2  2001      3          Alabama  all sectors   5.87  345.77802  5894.61038   
3  2001      4          Alabama  all sectors   5.72  347.18634  6064.53539   
4  2001      5          Alabama  all sectors   5.60  359.09236  6413.96530   

   tavg  
0  43.7  
1  55.0  
2  53.4  
3  66.0  
4  72.2  


In [5]:
load = pd.read_csv('../datasets/merged_electricity_temperature_data.csv')
load.shape

(13850, 8)